# Neural Networks

In [59]:
import time
import os
import pprint
import torch
import torch.nn as nn
from sklearn.metrics import ConfusionMatrixDisplay
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import librosa.display
import tqdm.notebook as tq
import utils
from pydub import AudioSegment
from tkinter import Tcl # file sorting by name

# Load STFT Dataset

### Dictionary creation for the classes

We want a dictionary indicating a numbeer for each genre:

{0: 'Hip-Hop', 1: 'Pop', 2: 'Folk', 3: 'Rock', 4: 'Experimental', 5: 'International', 6: 'Electronic', 7: 'Instrumental'}

### Creation of the labels vector

In [60]:
def create_single_dataset(folder_path, tracks_dataframe, genre_dictionary):    
    labels = []
   
    _, file_list = get_sorted_file_paths(folder_path)
    
    for i,file in enumerate(file_list):
        #print("considering file:",file, "({}/{})".format(i,len(file_list)))
        track_id_clip_id = file.split('.')[0]
        track_id = track_id_clip_id.split('_')[0]
        #print("track id with clip: {}, track id: {}".format(track_id_clip_id, track_id))
        genre = tracks_dataframe.loc[int(track_id)]
        #print("genre from dataframe: ", genre)
        label = genre_dictionary[genre]
        #print("label from dictionary:",label)
        labels.append(label)
    print("labels length: {}".format(len(labels)))
    return labels
    

#create the train,validation and test vectors using the files in the train/validation/test folders
def create_dataset_splitted(folder_path):
    train_folder = os.path.join(folder_path,'train') # concatenate train folder to path
    validation_folder = os.path.join(folder_path,'validation') # concatenate train folder to path
    test_folder = os.path.join(folder_path,'test') # concatenate train folder to path
    
    print("train_folder:",train_folder)
    print("validation_folder:",validation_folder)
    print("test_folder:",test_folder,"\n")
    
    AUDIO_DIR = os.environ.get('AUDIO_DIR')
    print("audio directory: ",AUDIO_DIR)
    print("Loading tracks.csv...")
    tracks = utils.load('data/fma_metadata/tracks.csv')
    
    #get only the small subset of the dataset
    small = tracks[tracks['set', 'subset'] <= 'small']
    print("small dataset shape:",small.shape)    

    small_training = small.loc[small[('set', 'split')] == 'training']['track']
    small_validation = small.loc[small[('set', 'split')] == 'validation']['track']
    small_test = small.loc[small[('set', 'split')] == 'test']['track']

    print("Track.csv: {} training samples, {} validation samples, {} test samples\n".format(len(small_training), len(small_validation), len(small_test)))

    small_training_top_genres = small_training['genre_top']
    small_validation_top_genres = small_validation['genre_top']
    small_test_top_genres = small_test['genre_top']
    
    #create dictionary of genre classes:
    unique_genres = small_training_top_genres.unique()
    unique_genres = np.array(unique_genres)
    print("there are {} unique genres".format(len(unique_genres)))
    genre_dictionary = {}
    for i,genre in enumerate(unique_genres):
        genre_dictionary[genre] = i
    print("Dictionary of genres created:",genre_dictionary)
    
    
    Y_train = create_single_dataset(train_folder, small_training_top_genres, genre_dictionary)
    Y_validation = create_single_dataset(validation_folder, small_validation_top_genres, genre_dictionary)
    Y_test = create_single_dataset(test_folder, small_test_top_genres, genre_dictionary)
    
    return Y_train, Y_validation, Y_test
 
def get_sorted_file_paths(folder_path):
    file_list = os.listdir(folder_path)
    #sort the dataset files in alphabetical order (important to associate correct labels created using track_id in track.csv)
    file_list = Tcl().call('lsort', '-dict', file_list) # sort file by name: 2_0,2_1, ... 2_9,3_0, ... 400_0,400_1, ...
    file_paths = [os.path.join(folder_path, file_name) for file_name in file_list] #join filename with folder path
    #print("There are {} in the folder: {}".format(len(file_list),file_list))
    return file_paths, file_list
    
    
folder_path="data/fma_small_stft_transposed_22050_overlapped"
Y_train, Y_validation, Y_test = create_dataset_splitted(folder_path)

train_folder: data/fma_small_stft_transposed_22050_overlapped/train
validation_folder: data/fma_small_stft_transposed_22050_overlapped/validation
test_folder: data/fma_small_stft_transposed_22050_overlapped/test 

audio directory:  ./data/fma_small/
Loading tracks.csv...
small dataset shape: (8000, 52)
Track.csv: 6400 training samples, 800 validation samples, 800 test samples

there are 8 unique genres
Dictionary of genres created: {'Hip-Hop': 0, 'Pop': 1, 'Folk': 2, 'Rock': 3, 'Experimental': 4, 'International': 5, 'Electronic': 6, 'Instrumental': 7}
labels length: 127940
labels length: 16000
labels length: 16000


# Dataset Class

Class to load the STFT from files. Each file has a (128,513) matrix containing the STFT of a 3 seconds audio clip.

In [61]:
# Define the custom class for accessing our dataset
class MyDataset(Dataset):
    def __init__(self, file_list, labels, transform=None, verbose = False):
        self.file_list = file_list
        self.labels=labels
        self.transform = transform
        self.verbose = verbose

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        # returns a training sample and its label
        file_path = self.file_list[idx]
        label = torch.tensor(self.labels[idx])
        stft_vector = torch.tensor(np.load(file_path)) #load from file
        
        # Normalize your data here
        if self.transform:
            if(self.verbose==True):
                print("TRANSFORM: applying transform to tensor shape:",stft_vector.shape,"content:",stft_vector)
            stft_vector = self.transform(torch.unsqueeze(stft_vector, dim=0)) #unsqueeze needed for the torchvision normalize method
            if(self.verbose==True):
                print("TRANSFORM: after transform shape:",stft_vector.shape,"content:",stft_vector)
            stft_vector = torch.squeeze(stft_vector, dim=0)
            if(self.verbose==True):
                print("TRANSFORM: after squeeze shape:",stft_vector.shape,"content:",stft_vector)

        
        return stft_vector, label

In [62]:
folder_path="data/fma_small_stft_transposed_22050_overlapped"

train_folder = os.path.join(folder_path,'train') # concatenate train folder to path
validation_folder = os.path.join(folder_path,'validation') # concatenate train folder to path
test_folder = os.path.join(folder_path,'test') # concatenate train folder to path

train_file_paths, _ = get_sorted_file_paths(train_folder)
train_dataset = MyDataset(train_file_paths, Y_train)
print("len of train dataset: ",len(train_dataset))

validation_file_paths, _ = get_sorted_file_paths(validation_folder)
validation_dataset = MyDataset(validation_file_paths, Y_validation)
print("len of validation dataset: ",len(validation_file_paths))

test_file_paths, _ = get_sorted_file_paths(test_folder)
test_dataset = MyDataset(test_file_paths, Y_test)
print("len of test dataset: ",len(test_dataset))

len of train dataset:  127940
len of validation dataset:  16000
len of test dataset:  16000


# Data normalization
We will use Z-Score to normalize the training, validation and test set by calculating the mean and the std deviation on the training set.

In [63]:
save_filename = './data/fma_small_stft_transposed_22050_overlapped/train_mean'
std_save_filename = './data/fma_small_stft_transposed_22050_overlapped/train_std_deviation'

## Calculation of mean and standard deviation (Long)

In [10]:
batch_size=1
total_n_batches = len(train_dataset)/batch_size
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
current_sum=0

#iter all the training set by batches and calculate the sum of all the sample values (513*128 values for each sample)
for batch_idx, batch in enumerate(train_loader):
    #print("batch",batch_idx,"/",total_n_batches,"current_sum:",current_sum)
    inputs = batch[0]
    labels = batch[1]
    #print("inputs: shape:",inputs.shape,"content:",inputs)
    #print("labels:",labels)
    for sample in inputs:
        #print("sample: shape",sample.shape,"content:",sample)
        current_sum += torch.sum(sample)
        #print("current_sum:",current_sum)
print("final sum:",current_sum)



final sum: tensor(8.9296e+09)


In [11]:
mean = current_sum/(len(train_dataset)*513*128) #divide the sum for the total number of values considerated
print("mean of training set:",mean)

print("Saving the mean in file:",save_filename) 
np.save(save_filename,mean)

mean of training set: tensor(1.0629)
Saving the mean in file: ./data/fma_small_stft_transposed_22050_overlapped/train_mean


In [13]:
#now let's calculate the standard deviation (squared root of the variance)

batch_size=1
total_n_batches = len(train_dataset)/batch_size
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
current_sum_of_squares = 0

for batch_idx, batch in enumerate(train_loader):
    #print("batch",batch_idx,"/",total_n_batches,"current_sum_of_squares:",current_sum_of_squares)
    inputs = batch[0]
    labels = batch[1]
    #print("inputs: shape:",inputs.shape,"content:",inputs)
    #print("labels:",labels)
    for sample in inputs:
        #print("sample shape",sample.shape)
        for row in sample:
            #print("row shape:",row.shape)
            for elem in row:
                #print("elem: shape",elem.shape,"content:",elem)
                difference = elem - mean
                difference_squared = difference**2
                current_sum_of_squares += difference_squared
                #print("current_sum:",current_sum)
print("final sum of squares:",current_sum_of_squares)


final sum of squares: tensor(7.8299e+10)


In [7]:
import math

variance = current_sum_of_squares/((len(train_dataset) * 513 * 128)-1)
std_deviation = math.sqrt(variance)

print("variance:",variance)
print("std_deviation:",std_deviation)

print("Saving the std_deviation in file:",std_save_filename) 
np.save(std_save_filename,std_deviation)

NameError: name 'current_sum_of_squares' is not defined

## Load calculated mean and std deviation from file

In [64]:
loaded_mean = np.load(save_filename+'.npy')
print("loaded mean:",loaded_mean)

loaded_std = np.load(std_save_filename+'.npy')
print("loaded std:",loaded_std)

loaded mean: 1.0629134
loaded std: 3.0528938510507846


## Create the normalized dataset

In [65]:
from torchvision import transforms

batch_size = 1

transform = transforms.Compose([
    transforms.Normalize(mean= loaded_mean, std= loaded_std)
])

train_dataset = MyDataset(train_file_paths, Y_train,  transform = transform)
validation_dataset = MyDataset(validation_file_paths, Y_validation,  transform = transform)
test_dataset = MyDataset(test_file_paths, Y_test,  transform = transform)

# Network Architecture Definition (nnet1)

In [10]:
class NNet1(nn.Module):
    def __init__(self):
        super(NNet1, self).__init__()

        self.conv1 = nn.Conv2d(1, 128, kernel_size=(4, 513))
        self.bn1 = nn.BatchNorm2d(128)
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 1))
        self.conv2 = nn.Conv2d(128, 128, kernel_size=(4, 1))
        self.bn2 = nn.BatchNorm2d(128)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 1))
        self.conv3 = nn.Conv2d(128, 256, kernel_size=(4, 1))
        self.bn3 = nn.BatchNorm2d(256)
        self.avgpool = nn.AvgPool2d(kernel_size=(26, 1))
        self.maxpool = nn.MaxPool2d(kernel_size=(26, 1))
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.2)
        self.dense1 = nn.Linear(512, 300)
        self.bn4 = nn.BatchNorm1d(300)
        self.dense2 = nn.Linear(300,150)
        self.bn5 = nn.BatchNorm1d(150)
        self.dense3 = nn.Linear(150, 8)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x.float())
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x_avg = self.avgpool(x)
        x_max = self.maxpool(x)
        x = torch.cat([x_avg, x_max], dim=1)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.dropout(x)
        x = self.bn5(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.softmax(x)
        return x

In [11]:
class NNet2(nn.Module):
    def __init__(self):
        super(NNet2, self).__init__()
        
        self.drop=nn.Dropout(0.2)
        self.conv1 = nn.Conv2d(1, 128, kernel_size=(4, 513))
        self.batch1=nn.BatchNorm2d(128)
        self.conv2 = nn.Conv2d(128,128, kernel_size=(4, 1),padding=1)
        self.batch2=nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=(4, 1),padding=2)
        self.batch3=nn.BatchNorm2d(128)
        self.maxpool = nn.MaxPool2d(kernel_size=(128,1))
        self.avgpool = nn.AvgPool2d(kernel_size=(128,1))
        self.fc1 = nn.Linear(250,128)
        self.bn1=nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128,64)
        self.bn2=nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 8)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.conv1(x.float())
        x = self.batch1(x)
        x = torch.relu(x)
        y = x
        x = self.conv2(x)
        x = self.batch2(x)
        x = torch.relu(x)
        x = self.conv3(x)
        x = self.batch3(x)
        x = torch.relu(x)
        # Sum between the first and third conv layers
        x = x[:, :, :, 0] + y[:, :, :, 0]
        
        x = torch.relu(x)
        x_max = self.maxpool(x)
        x_avg = self.avgpool(x)
        x = torch.cat([x_avg, x_max], dim=1)
        # Flatten the tensor for fully connected layers
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.drop(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.fc2(x)        
        x = self.drop(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = self.softmax(x)
        
        return x

# Hyperparameters

In [66]:
BATCH_SIZE=32
EPOCHS=10
LEARNING_RATE=0.0001

learning_rate_list = [0.0001,0.00001]
batch_size_list = [128,256,512]
reg_list=[0.001,0.0001,0.00001]

# Train function

In [67]:
def test(model, validation_dataset, Y_validation):
    # Stop parameters learning
    model.eval()

    validation_loader = torch.utils.data.DataLoader(validation_dataset)

    criterion = nn.CrossEntropyLoss()
    correct = 0
    total = 0
    total_loss = 0
    #confusion_matrix = np.zeros((8, 8), dtype=int)

    with torch.no_grad():
        for sample, label in validation_loader:
            
            sample = sample.unsqueeze(1)

            # Predict label
            output = model(sample)
            
            # Compute loss
            loss = criterion(output, label)
            total_loss += loss.item()

            max_index = torch.argmax(output).item()  # The index with maximum probability

            #confusion_matrix[label][max_index] += 1

            correct += (max_index == label)

    #cm = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix)
    #cm.plot()
    #print(confusion_matrix)
    accuracy = 100 * correct / len(Y_validation)
    average_loss = total_loss / len(Y_validation)

    model.train()
    return accuracy, average_loss

In [68]:
def train(model, dataset, batch_size, num_epochs, learning_rate, verbose = False, RGB=False, reg=1e-5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    val_loss_list=[]
    val_acc_list=[]
    train_loss_list=[]
    train_acc_list=[]
    counted_labels=[0,0,0,0,0,0,0,0]
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg)
    criterion = nn.CrossEntropyLoss()

    if not isinstance(dataset, Dataset):
        raise ValueError("The dataset parameter should be an instance of torch.utils.data.Dataset.")

    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    num_batches = len(data_loader)
    
    
    for epoch in range(num_epochs):
        running_loss = 0.0 
        running_accuracy = 0.0
        #initialize correctly predicted samples
        
        # Initialize the progress bar
        progress_bar = tq.tqdm(total=num_batches, unit="batch")
    
        # Initialize the progress bar description
        progress_bar.set_description(f"Epoch {epoch+1}/{num_epochs}")
        start_time = time.time()
        
        for batch_idx, batch in enumerate(data_loader):
            
            correct = 0 # reset train accuracy each batch
            
            inputs,labels = batch[0],batch[1]
            if(verbose == True):
                print("\ninputs shape:",inputs.size(),", dtype:",inputs.dtype," content: ",inputs)
                print("min value:",torch.min(inputs))
                print("max value:",torch.max(inputs))
                print("\nlabels shape:",labels.size(),",dtype:",labels.dtype,", content: ",labels)
            if(RGB==False):
                inputs = inputs.unsqueeze(1) #add a dimension if input is to be considered just grayscale
                #if input is RGB, there are already 3 channels
            
            # Extract the inputs and targets
            optimizer.zero_grad()
            outputs = model(inputs)
            
            if(verbose == True):
                print("\noutputs size:",outputs.size(),"content:",outputs)
                print("List of labels until now:",counted_labels)

            loss = criterion(outputs, labels) #labels need to be a vector of class indexes (0-7) of dim (batch_size)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            #calculate train accuracy
            for index, output in enumerate(outputs):
                max_index = torch.argmax(output).item() #the index with maximum probability
                counted_labels[labels[index].item()]+=1
                if(labels[index].item() == max_index):
                    correct += 1
            
                if(verbose==True):
                    print("considering output at index {}:".format(index,output))
                    print("max output index = {}",max_index)
                    if(labels[index].item() == max_index):
                        print("correct! in fact labels[index] = {}, max_index = {}".format(labels[index].item(),max_index))
                    else:
                        print("NOT correct! in fact labels[index] = {}, max_index = {}".format(labels[index].item(),max_index))

            
            accuracy = 100 * correct / batch_size
            running_accuracy += accuracy #epoch running_accuracy
            
            # Update the progress bar description and calculate bps
            #progress_bar.set_postfix({"Loss": running_loss / (batch_idx + 1)})
            average_accuracy = running_accuracy / (batch_idx + 1)
            average_loss = running_loss / (batch_idx + 1)
            progress_bar.set_postfix({"avg_loss": average_loss, "acc": accuracy, "avg_acc": average_accuracy})

            # Update the progress bar
            progress_bar.update(1)
            # Evaluate the model on the validation dataset
        
        #calculate train loss and accuracy
        average_loss = running_loss / len(data_loader)
        average_accuracy = running_accuracy / len(data_loader)
        train_loss_list.append(average_loss)
        train_acc_list.append(average_accuracy)
        
        #calculate validation loss and accuracy
        val_acc, val_loss = test(model, validation_dataset, Y_validation)
        val_loss_list.append(val_loss)
        val_acc_list.append(val_acc)
        
        
        print(f"Epoch [{epoch+1}/{num_epochs}],Train Loss: {average_loss:.4f}. Train Accuracy: {average_accuracy} Val Loss: {val_loss} Val Accuracy: {val_acc}")
        progress_bar.close()
    return train_loss_list, train_acc_list, val_loss_list, val_acc_list

# Network Architecture Definition (nnet1 + BN)

In [69]:
class NNet1_Small(nn.Module):
    def __init__(self):
        super(NNet1_Small, self).__init__()

        self.conv1 = nn.Conv2d(1, 64, kernel_size=(2, 513))
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(4, 1))
        self.conv2 = nn.Conv2d(64,128, kernel_size=(2, 1))
        self.bn2 = nn.BatchNorm2d(128)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(4, 1))
        self.conv3 = nn.Conv2d(128,64, kernel_size=(4, 1))
        self.bn3 = nn.BatchNorm2d(64)
        self.avgpool = nn.AvgPool2d(kernel_size=(2, 1))
        self.maxpool = nn.MaxPool2d(kernel_size=(2, 1))
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.3)
        self.dense1 = nn.Linear(256, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.dense2 = nn.Linear(64, 8)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x.float())
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x_avg = self.avgpool(x)
        x_max = self.maxpool(x)
        x = torch.cat([x_avg, x_max], dim=1)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.softmax(x)
        return x

In [15]:
model_NNet1_Small = NNet1_Small()
summary(model_NNet1_Small, (1, 128, 513))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 127, 1]          65,728
       BatchNorm2d-2           [-1, 64, 127, 1]             128
              ReLU-3           [-1, 64, 127, 1]               0
         MaxPool2d-4            [-1, 64, 31, 1]               0
            Conv2d-5           [-1, 128, 30, 1]          16,512
       BatchNorm2d-6           [-1, 128, 30, 1]             256
              ReLU-7           [-1, 128, 30, 1]               0
         MaxPool2d-8            [-1, 128, 7, 1]               0
            Conv2d-9             [-1, 64, 4, 1]          32,832
      BatchNorm2d-10             [-1, 64, 4, 1]             128
        AvgPool2d-11             [-1, 64, 2, 1]               0
        MaxPool2d-12             [-1, 64, 2, 1]               0
          Flatten-13                  [-1, 256]               0
           Linear-14                   

In [122]:
#TODO: Add batch_size optimization loop
save_directory="./results/NNet1_Small/"

lr_list= [ 0.0001]
r_list=[0.0001]

for i in lr_list:
    for j in r_list:
        if(j!=1e-5 and j!=1e-6):
            filename=save_directory+"lr_"+"0"+str(i).split(".")[1]+"_reg_"+"0"+str(j).split(".")[1]
        elif(j==1e-5):
            filename=save_directory+"lr_"+"0"+str(i).split(".")[1]+"_reg_"+"00001"
        else:
            filename=save_directory+"lr_"+"0"+str(i).split(".")[1]+"_reg_"+"000001"
        print(filename)
        model = NNet1_Small()
        train_loss_list, train_acc_list, val_loss_list, val_acc_list =train(model, train_dataset, batch_size=128, num_epochs=10, learning_rate=i, reg=j)
        print("Trained with learning rate=",i," and with regularization term=",j)
        print("Loss:",val_loss_list)
        print("Accuracy:",val_acc_list)
        save_values=[val_loss_list,val_acc_list]
        np.savetxt(filename,save_values)

./results/NNet1_Small/lr_00001_reg_00001


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [1/10],Train Loss: 1.9128. Train Accuracy: 40.0125 Val Loss: 1.8593959403708578 Val Accuracy: tensor([45.8375])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [2/10],Train Loss: 1.7943. Train Accuracy: 50.90078125 Val Loss: 1.8144761948511003 Val Accuracy: tensor([47.6750])


...

Epoch [8/10],Train Loss: 1.6657. Train Accuracy: 61.46953125 Val Loss: 1.8000839565470814 Val Accuracy: tensor([47.2313])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [9/10],Train Loss: 1.6564. Train Accuracy: 62.5 Val Loss: 1.7685952556654811 Val Accuracy: tensor([50.5125])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [10/10],Train Loss: 1.6467. Train Accuracy: 63.62109375 Val Loss: 1.7865395892858504 Val Accuracy: tensor([48.4625])
Trained with learning rate= 0.0001  and with regularization term= 0.0001
Loss: [1.8593959403708578, 1.8144761948511003, 1.7945972227230669, 1.7790009344667197, 1.7896004073768854, 1.779199239976704, 1.7810167167559265, 1.8000839565470814, 1.7685952556654811, 1.7865395892858504]
Accuracy: [tensor([45.8375]), tensor([47.6750]), tensor([48.6312]), tensor([49.8688]), tensor([48.6375]), tensor([49.3750]), tensor([48.8063]), tensor([47.2313]), tensor([50.5125]), tensor([48.4625])]


In [71]:
#TODO: Add batch_size optimization loop
save_directory="./best_models/"

learning_rate_list = [0.001]
batch_size_list = [128,256,512]
reg_list=[0.00001]

for i in learning_rate_list:
    for j in reg_list:
        filename=save_directory+"results/NNet1_Small"        
        print(filename)
        model = NNet1_Small()
        train_loss_list, train_acc_list, val_loss_list, val_acc_list =train(model, train_dataset, batch_size=64, num_epochs=10, learning_rate=i, reg=j)
        print("Trained with learning rate=",i," and with regularization term=",j)
        print("Loss:",val_loss_list)
        print("Accuracy:",val_acc_list)
        save_values=[ train_loss_list, train_acc_list, val_loss_list,val_acc_list]
        np.savetxt(filename,save_values)
        torch.save(model.state_dict(), save_directory+"models/NNet1_Small")

./best_models/results/NNet1_Small


  0%|          | 0/2000 [00:00<?, ?batch/s]

Epoch [1/10],Train Loss: 1.8076. Train Accuracy: 46.7859375 Val Loss: 1.815145688265562 Val Accuracy: tensor([44.8750])


  0%|          | 0/2000 [00:00<?, ?batch/s]

Epoch [2/10],Train Loss: 1.7305. Train Accuracy: 54.23203125 Val Loss: 1.8216929790973664 Val Accuracy: tensor([44.4750])


...

Epoch [8/10],Train Loss: 1.6398. Train Accuracy: 63.2359375 Val Loss: 1.8199008042141795 Val Accuracy: tensor([44.9313])


  0%|          | 0/2000 [00:00<?, ?batch/s]

Epoch [9/10],Train Loss: 1.6301. Train Accuracy: 64.175 Val Loss: 1.7777148803249 Val Accuracy: tensor([49.1500])


  0%|          | 0/2000 [00:00<?, ?batch/s]

Epoch [10/10],Train Loss: 1.6235. Train Accuracy: 64.859375 Val Loss: 1.7791675435304641 Val Accuracy: tensor([48.9125])
Trained with learning rate= 0.001  and with regularization term= 1e-05
Loss: [1.815145688265562, 1.8216929790973664, 1.777285971775651, 1.7788502685800194, 1.8065976762101055, 1.7716257748901845, 1.7790606408789753, 1.8199008042141795, 1.7777148803249, 1.7791675435304641]
Accuracy: [tensor([44.8750]), tensor([44.4750]), tensor([49.3688]), tensor([48.8625]), tensor([46.3187]), tensor([49.6875]), tensor([48.9062]), tensor([44.9313]), tensor([49.1500]), tensor([48.9125])]


In [123]:
#TODO: Add batch_size optimization loop
save_directory="./results/NNet1/"

lr_list= [ 0.001]
r_list=[0.0001]

for i in lr_list:
    for j in r_list:
        if(j!=1e-5 and j!=1e-6):
            filename=save_directory+"lr_"+"0"+str(i).split(".")[1]+"_reg_"+"0"+str(j).split(".")[1]
        elif(j==1e-5):
            filename=save_directory+"lr_"+"0"+str(i).split(".")[1]+"_reg_"+"000001"
        else:
            filename=save_directory+"lr_"+"0"+str(i).split(".")[1]+"_reg_"+"0000001"
        print(filename)
        model = NNet1()
        train_loss_list, train_acc_list, val_loss_list, val_acc_list =train(model, train_dataset, batch_size=128, num_epochs=10, learning_rate=i, reg=j)
        print("Trained with learning rate=",i," and with regularization term=",j)
        print("Loss:",val_loss_list)
        print("Accuracy:",val_acc_list)
        save_values=[val_loss_list,val_acc_list]
        np.savetxt(filename,save_values)

./results/NNet1/lr_0001_reg_00001


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [1/10],Train Loss: 1.7846. Train Accuracy: 48.8125 Val Loss: 1.8186630289033054 Val Accuracy: tensor([45.0438])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [2/10],Train Loss: 1.7078. Train Accuracy: 56.38828125 Val Loss: 1.7965274090394379 Val Accuracy: tensor([47.3312])


...

Epoch [8/10],Train Loss: 1.6059. Train Accuracy: 66.7046875 Val Loss: 1.795549453318119 Val Accuracy: tensor([47.0687])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [9/10],Train Loss: 1.5951. Train Accuracy: 67.81796875 Val Loss: 1.8151078140363097 Val Accuracy: tensor([45.2000])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [10/10],Train Loss: 1.5874. Train Accuracy: 68.5265625 Val Loss: 1.817907063394785 Val Accuracy: tensor([45.0563])
Trained with learning rate= 0.001  and with regularization term= 0.0001
Loss: [1.8186630289033054, 1.7965274090394379, 1.8315542057305574, 1.7864047286510467, 1.8000281608402728, 1.800094104759395, 1.791915791131556, 1.795549453318119, 1.8151078140363097, 1.817907063394785]
Accuracy: [tensor([45.0438]), tensor([47.3312]), tensor([43.2938]), tensor([48.2188]), tensor([46.9125]), tensor([46.9250]), tensor([47.6937]), tensor([47.0687]), tensor([45.2000]), tensor([45.0563])]


In [21]:
#TODO: Add batch_size optimization loop
save_directory="./results/NNet2/"

learning_rate_list = [0.001]
batch_size_list = [128,256,512]
reg_list=[0.00001,0.000001]

for i in learning_rate_list:
    for j in reg_list:
        if(j!=1e-5 and j!=1e-6):
            filename=save_directory+"lr_"+"0"+str(i).split(".")[1]+"_reg_"+"0"+str(j).split(".")[1]
        elif(j==1e-5):
            filename=save_directory+"lr_"+"0"+str(i).split(".")[1]+"_reg_"+"000001"
        else:
            filename=save_directory+"lr_"+"0"+str(i).split(".")[1]+"_reg_"+"0000001"

       
        print(filename)
        model = NNet2()
        train_loss_list, train_acc_list, val_loss_list, val_acc_list =train(model, train_dataset, batch_size=128, num_epochs=10, learning_rate=i, reg=j)
        print("Trained with learning rate=",i," and with regularization term=",j)
        print("Loss:",val_loss_list)
        print("Accuracy:",val_acc_list)
        save_values=[val_loss_list,val_acc_list]
        np.savetxt(filename,save_values)

./results/NNet2/lr_0001_reg_000001


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [1/10],Train Loss: 1.9199. Train Accuracy: 34.6078125 Val Loss: 1.9418242857679724 Val Accuracy: tensor([30.8687])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [2/10],Train Loss: 1.8650. Train Accuracy: 40.284375 Val Loss: 1.9395233458429575 Val Accuracy: tensor([32.0500])


...

Epoch [8/10],Train Loss: 1.8198. Train Accuracy: 44.71875 Val Loss: 1.9319837843030692 Val Accuracy: tensor([32.9188])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [9/10],Train Loss: 1.8143. Train Accuracy: 45.28046875 Val Loss: 1.9393858048543333 Val Accuracy: tensor([32.5563])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [10/10],Train Loss: 1.8107. Train Accuracy: 45.72734375 Val Loss: 1.9426354933977128 Val Accuracy: tensor([32.1813])
Trained with learning rate= 0.001  and with regularization term= 1e-06
Loss: [1.931972404450178, 1.924639566987753, 1.9423593048751355, 1.9347730785906314, 1.9543162247166037, 1.9442311258390546, 1.9336775086149574, 1.9319837843030692, 1.9393858048543333, 1.9426354933977128]
Accuracy: [tensor([32.5750]), tensor([32.9813]), tensor([31.5063]), tensor([32.7875]), tensor([31.0562]), tensor([32.0250]), tensor([33.0375]), tensor([32.9188]), tensor([32.5563]), tensor([32.1813])]


# Raw audio (1D)

In the following section we'll implement a Neural Network based on the classification of the raw audio. The raw audio is an array containing the amplitude of the audio wave for each sample of a 3 second clip (We used a sampling rate of 22050 Hz). Each entry has a size of (1,66150).

## Raw audio dataset

In [22]:
class MyDatasetRaw(Dataset):
    def __init__(self, file_list, labels, transform=None, verbose=False):
        self.file_list = file_list
        self.labels=labels
        self.transform = transform
        self.verbose=verbose

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = self.file_list[idx]
        label = torch.tensor(self.labels[idx])
        raw_vector = np.load(file_path).astype(np.int16) # Ensure int16 data type
        if(self.verbose==True):
            print("raw vector shape:",raw_vector.shape)
        raw_vector = torch.tensor(raw_vector)
        
        # Normalize your data here
        if self.transform:
            
            #convert to float64 tensor
            raw_vector = raw_vector.double()
            if(self.verbose==True):
                print("TRANSFORM: applying transform to tensor shape:",raw_vector.shape,"content:",raw_vector)
            raw_vector = torch.unsqueeze(raw_vector, dim=0)
            #print("TRANSFORM: after first unsqueeze:",raw_vector.shape,"content:",raw_vector)
            raw_vector = torch.unsqueeze(raw_vector, dim=0) #unsqueeze two times (needed for torchvision normalize method)
            #print("TRANSFORM: after second unsqueeze:",raw_vector.shape,"content:",raw_vector)
            raw_vector = self.transform(raw_vector) #normalize the sample
            if(self.verbose==True):
                print("TRANSFORM: after transform shape:",raw_vector.shape,"content:",raw_vector)
            raw_vector = torch.squeeze(raw_vector, dim=0)
            raw_vector = torch.squeeze(raw_vector, dim=0)
            if(self.verbose==True):
                print("TRANSFORM: after double squeeze shape:",raw_vector.shape,"content:",raw_vector)
        
        return raw_vector, label        

# Normalization of raw audio

We calculate mean and std deviation

In [23]:
mean_save_filename_raw = './data/fma_small_raw_array_22050_overlapped/train_mean'
std_save_filename_raw = './data/fma_small_raw_array_22050_overlapped/train_std_deviation'

# Calculation of mean raw (Long)

In [ ]:
train_dataset = MyDatasetRaw(file_paths_train, Y_train)
batch_size=1
total_n_batches = len(train_dataset)/batch_size
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
current_sum=0

#iter all the training set by batches and calculate the sum of all the sample values (513*128 values for each sample)
for batch_idx, batch in enumerate(train_loader):
    if(batch_idx%1000==0):
        print("batch",batch_idx,"/",total_n_batches,"(",round((batch_idx/len(train_dataset)*100)),"%), current_sum:",current_sum)
    
    inputs = batch[0]
    labels = batch[1]
    #print("inputs: shape:",inputs.shape,"content:",inputs)
    #print("labels:",labels)
    for sample in inputs:
        #print("sample: shape",sample.shape,"content:",sample)
        current_sum += torch.sum(sample)
       
        #print("type of current_sum:",current_sum.dtype)
        #print("current_sum:",current_sum)
print("final sum:",current_sum)

In [ ]:
print("current_sum",current_sum)
mean_raw = current_sum/(len(train_dataset)*66150) #divide the sum for the total number of values considerated
print("mean of training set:",mean_raw)

print("Saving the mean in file:",mean_save_filename_raw) 
np.save(mean_save_filename_raw,mean_raw)

# Calculation of std deviation raw (Long)

In [ ]:
#now let's calculate the standard deviation (squared root of the variance)

batch_size=1
total_n_batches = len(train_dataset)/batch_size
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
current_sum_of_squares = 0

for batch_idx, batch in enumerate(train_loader):
    if(batch_idx%1000==0):
        print("batch",batch_idx,"/",total_n_batches,round((batch_idx/len(train_dataset)*100)),"%, current_sum_of_squares:",current_sum_of_squares)
    inputs = batch[0]
    labels = batch[1]
    #print("inputs: shape:",inputs.shape,"content:\n",inputs)
    #print("labels:",labels)
    for elem in inputs:
        elem = elem.double() #convert to float64 for precise calculations
        #print("elem: shape",elem.shape,"content:\n",elem)
        difference = elem - mean_raw
        #print("difference: shape:",difference.shape,"content:\n", difference)
        difference_squared = difference**2
        #print("difference_squared: shape:",difference_squared.shape,"content:\n", difference_squared)
        current_sum_of_squares += torch.sum(difference_squared)
        #print("current_sum_of_squares:",current_sum_of_squares)
print("final sum of squares:",current_sum_of_squares)

In [ ]:
import math

variance_raw = current_sum_of_squares/((len(train_dataset)*66150)-1)
std_deviation_raw = math.sqrt(variance)

print("variance raw:",variance)
print("std_deviation_raw:",std_deviation_raw)

print("Saving the std_deviation_raw in file:",std_save_filename_raw) 
np.save(std_save_filename_raw,std_deviation_raw)

# Load mean and std from file (fast)

In [24]:
loaded_mean_raw = np.load(mean_save_filename_raw+'.npy')
print("loaded mean:",loaded_mean_raw)

loaded_std_raw = np.load(std_save_filename_raw+'.npy')
print("loaded std:",loaded_std_raw)

loaded mean: -16.984083
loaded std: 1032.9510216986293


## Create dataset for raw audio

The labels are the same as the STFT dataset (in the same order).

In [25]:
folder_path="data/fma_small_raw_array_22050_overlapped"

train_folder = os.path.join(folder_path,'train') # concatenate train folder to path
validation_folder = os.path.join(folder_path,'validation') # concatenate train folder to path
test_folder = os.path.join(folder_path,'test') # concatenate train folder to path

train_file_paths, _ = get_sorted_file_paths(train_folder)
train_dataset = MyDatasetRaw(train_file_paths, Y_train)
print("len of train dataset: ",len(train_dataset))

validation_file_paths, _ = get_sorted_file_paths(validation_folder)
validation_dataset = MyDatasetRaw(validation_file_paths, Y_validation)
print("len of validation dataset: ",len(validation_file_paths))

test_file_paths, _ = get_sorted_file_paths(test_folder)
test_dataset = MyDatasetRaw(test_file_paths, Y_test)
print("len of test dataset: ",len(test_dataset))

len of train dataset:  127940
len of validation dataset:  16000
len of test dataset:  16000


# Neural Network Architecture for raw audio

We implemented a lightweight CNN to classify the samples with their raw audio.

In [26]:
class NNet_Raw(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(NNet_Raw, self).__init__()
        
        self.conv1 = nn.Conv1d(1, 32, kernel_size=16)
        self.conv2 = nn.Conv1d(32, 8, kernel_size=16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=32)
        self.maxpool1 = nn.MaxPool1d(kernel_size=8)
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.batchnorm2 = nn.BatchNorm1d(8)
        self.batchnorm3 = nn.BatchNorm1d(24)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(248, 24)
        self.fc3 = nn.Linear(24, 8)
        self.softmax=nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.maxpool1(x.float())
        x = self.conv1(x)
        x = self.relu(x)
        x = self.batchnorm1(x)
        x=self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.batchnorm2(x)     
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.batchnorm3(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

In [56]:
#TODO: Add batch_size optimization loop
save_directory="./best_models/"

learning_rate_list = [0.001]
batch_size_list = [128,256,512]
reg_list=[0.0001]

for i in learning_rate_list:
    for j in reg_list:
        filename=save_directory+"results/NNet_Raw"        
        print(filename)
        model = NNet_Raw()
        train_loss_list, train_acc_list, val_loss_list, val_acc_list =train(model, train_dataset, batch_size=128, num_epochs=10, learning_rate=i, reg=j)
        print("Trained with learning rate=",i," and with regularization term=",j)
        print("Loss:",val_loss_list)
        print("Accuracy:",val_acc_list)
        save_values=[ train_loss_list, train_acc_list, val_loss_list,val_acc_list]
        np.savetxt(filename,save_values)
        torch.save(model.state_dict(), save_directory+"models/NNet_Raw")

./best_models/results/NNet_Raw


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [1/10],Train Loss: 1.9130. Train Accuracy: 36.08515625 Val Loss: 1.9141093373969198 Val Accuracy: tensor([34.8312])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [2/10],Train Loss: 1.8659. Train Accuracy: 40.35859375 Val Loss: 1.8956503313481807 Val Accuracy: tensor([37.2125])


...

Epoch [8/10],Train Loss: 1.8224. Train Accuracy: 44.6875 Val Loss: 1.8739052537232637 Val Accuracy: tensor([38.9688])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [9/10],Train Loss: 1.8179. Train Accuracy: 45.11796875 Val Loss: 1.8722335183471441 Val Accuracy: tensor([39.2125])


  0%|          | 0/1000 [00:00<?, ?batch/s]

Epoch [10/10],Train Loss: 1.8131. Train Accuracy: 45.55546875 Val Loss: 1.8769738131389022 Val Accuracy: tensor([38.6000])
Trained with learning rate= 0.001  and with regularization term= 0.0001
Loss: [1.9141093373969198, 1.8956503313481807, 1.8685628568083048, 1.873325175538659, 1.8723637982606889, 1.8777673560529948, 1.855006039455533, 1.8739052537232637, 1.8722335183471441, 1.8769738131389022]
Accuracy: [tensor([34.8312]), tensor([37.2125]), tensor([39.6875]), tensor([39.0438]), tensor([39.3937]), tensor([38.9188]), tensor([41.2812]), tensor([38.9688]), tensor([39.2125]), tensor([38.6000])]


In [ ]:
MyModel=NNet_Raw()
summary(MyModel, [1,66150])
print(MyModel)

In [58]:
train_loss_list, train_acc_list, val_loss_list, val_acc_list =train(MyModel, train_dataset, batch_size=128, num_epochs=10, learning_rate=0.01, verbose=False)

NameError: name 'MyModel' is not defined

# Results Loader

In [72]:
def find_best_value(values,loss):
    if loss==True:
        max_v=100
    else:
        max_v=0
    index=-1
    for i in range(len(values)):
        if loss==True:
            if values[i]<max_v:
                max_v=values[i]
                index=i+1
        else:
            if values[i]>max_v:
                max_v=values[i]
                index=i+1
    return max_v,index
            

In [73]:
res_directory="./results/"
models=os.listdir(res_directory) 
for i in models:
    best_loss=100
    best_acc=0
    best_loss_ep=0
    best_acc_ep=0
    model_folder=res_directory+i
    trials=os.listdir(model_folder)
    if(len(trials)==0):
        continue
    best_trials=[(best_loss,best_loss_ep),(best_acc,best_acc_ep)]
    best_trials_names=['','']
    for j in trials:
        res=np.loadtxt(model_folder+"/"+j)
        loss,epoch_l=find_best_value(res[0],True)
        accuracy,epoch_a=find_best_value(res[1],False)
        if(loss<best_loss):
            best_trials_names[0]=j
            best_loss=loss
            best_loss_ep=epoch_l
            best_trials[0]=(best_loss,best_loss_ep)
        if(accuracy>best_acc):
            best_trials_names[1]=j
            best_acc=accuracy
            best_acc_ep=epoch_a
            best_trials[1]=(best_acc,best_acc_ep)
            
    print("Model:",i)
    print("Best Model for accuracy:",best_trials_names[1])
    print("Value:",best_trials[1][0],"Epoch:",best_trials[1][1])
    print("Best Model for loss:",best_trials_names[0])
    print("Value:",best_trials[0][0],"Epoch:",best_trials[0][1])
    print("")
    
print(models)

Model: NNet_Raw
Best Model for accuracy: lr_0001_reg_00001
Value: 41.58124923706055 Epoch: 9
Best Model for loss: lr_0001_reg_00001
Value: 1.8506742839217185 Epoch: 9

Model: NNet1
Best Model for accuracy: lr_00001_reg_0001
Value: 50.9375 Epoch: 2
Best Model for loss: lr_0001_reg_000001
Value: 1.7595391278117896 Epoch: 4

Model: NNet1_Small
Best Model for accuracy: lr_00001_reg_00001
Value: 50.51250076293945 Epoch: 9
Best Model for loss: lr_0001_reg_000001
Value: 1.7661717012748122 Epoch: 10

Model: Ensemble
Best Model for accuracy: lr_0001_reg_000001
Value: 50.76874923706055 Epoch: 2
Best Model for loss: lr_0001_reg_000001
Value: 1.769827719859779 Epoch: 2

Model: NNet2
Best Model for accuracy: lr_00001_reg_000001
Value: 38.79375076293945 Epoch: 9
Best Model for loss: lr_00001_reg_000001
Value: 1.8803115216344595 Epoch: 9

['Transfer_Learning', 'NNet_Raw', 'NNet1', 'NNet1_Small', 'Ensemble', 'NNet2', 'Ensemble_No_Weights']


# NN on echonest features 

As you can see below, unfortunately we cannot perform a train using echonest features (danceability, tempo, acousticness, ...) because there are only ►1300 tracks with echonest features which are not NaN.

In [26]:
print("opening csvs...")

tracks = utils.load('data/fma_metadata/tracks.csv')

echonest = utils.load('data/fma_metadata/echonest.csv')
echonest = echonest['echonest', 'audio_features']
small = tracks[tracks['set', 'subset'] <= 'small']

print("small dataset shape:",small.shape)
print("echonest csv shape (only audio features):",echonest.shape)

opening csvs...
small dataset shape: (8000, 52)
echonest csv shape (only audio features): (13129, 8)


In [27]:
#select small dataset from echonest csv

track_ids = small.index.values.tolist()
print("Track ids shape:",len(track_ids),"content:",track_ids[:10],"...")
echonest_small = pd.DataFrame(echonest,index=track_ids)

ipd.display(echonest_small)

Track ids shape: 8000 content: [2, 5, 10, 140, 141, 148, 182, 190, 193, 194] ...


,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590
140,0.376312,0.734079,0.265685,0.669581,0.085995,0.039068,107.952,0.609991
141,0.963657,0.435933,0.075632,0.345493,0.105686,0.026658,33.477,0.163950
...,...,...,...,...,...,...,...,...
154308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
X_train_echonest = echonest_small.to_numpy(dtype=np.float16)
print("X_train_echonest: shape:",X_train_echonest.shape)

nan_rows = np.argwhere(np.isnan(X_train_echonest).all(axis=1))

print("There are",len(nan_rows),"rows containing NaN only as data")
#nan_rows = nan_rows.squeeze()
#for i in nan_rows:
#    print("row:",i,":",X_train_echonest[i])

X_train_echonest: shape: (8000, 8)
There are 6706 rows containing NaN only as data


# ResNet
We try to use transfer learning using a ResNet18 by torch

In [159]:
import torchvision

model_ResNet18 = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x0_5', pretrained=True)
summary(model_ResNet18, (3,128,513))


# Remove layers you want to discard

class MyModel_ResNet18(nn.Module):
    def __init__(self):
        super(MyModel_ResNet18, self).__init__()
        pretrained_model = torchvision.models.shufflenet_v2_x0_5(pretrained=True)
        layers=list(pretrained_model.children())[:-4]
        #print(layers)
        self.features = nn.Sequential(*layers)
        
        
        #self.conv1=nn.Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
        #self.relu1=nn.ReLU(inplace=True)
        #self.mp1=nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        #self.conv2=nn.Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        '''self.relu2=nn.ReLU(inplace=True)
        self.mp2=nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        self.conv3=nn.Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu3=nn.ReLU(inplace=True)
        self.conv4=nn.Conv2d(192, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu4=nn.ReLU(inplace=True)
        self.conv5=nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu5=nn.ReLU(inplace=True)
        self.mp5=nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)'''
        
        
        self.pool= nn.AdaptiveAvgPool2d(1)
        self.flatten=nn.Flatten() 
        self.last_layer = nn.Linear(49920, 32)
        self.fc= nn.Linear(32, 8)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        
        #x=self.conv1(x)
        #x=self.relu1(x)
        #x=self.mp1(x)
        '''x=self.conv2(x)
        x=self.relu2(x)
        x=self.mp2(x)    
        x=self.conv4(x)
        x=self.relu4(x)
        x=self.conv5(x)
        x=self.relu5(x)
        x=self.mp5(x)'''
        
        x = self.features(x)
        #x = self.pool(x)
        x = self.flatten(x)
        x = self.last_layer(x)
        x = self.softmax(x)
        return x
    





----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 24, 64, 257]             648
       BatchNorm2d-2          [-1, 24, 64, 257]              48
              ReLU-3          [-1, 24, 64, 257]               0
         MaxPool2d-4          [-1, 24, 32, 129]               0
            Conv2d-5           [-1, 24, 16, 65]             216
       BatchNorm2d-6           [-1, 24, 16, 65]              48
            Conv2d-7           [-1, 24, 16, 65]             576
       BatchNorm2d-8           [-1, 24, 16, 65]              48
              ReLU-9           [-1, 24, 16, 65]               0
           Conv2d-10          [-1, 24, 32, 129]             576
      BatchNorm2d-11          [-1, 24, 32, 129]              48
             ReLU-12          [-1, 24, 32, 129]               0
           Conv2d-13           [-1, 24, 16, 65]             216
      BatchNorm2d-14           [-1, 24,

Using cache found in /home/riccardo/.cache/torch/hub/pytorch_vision_v0.10.0


In [160]:
    
model = MyModel_ResNet18()
summary(model, (3,128,513))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 24, 64, 257]             648
       BatchNorm2d-2          [-1, 24, 64, 257]              48
              ReLU-3          [-1, 24, 64, 257]               0
         MaxPool2d-4          [-1, 24, 32, 129]               0
            Conv2d-5           [-1, 24, 16, 65]             216
       BatchNorm2d-6           [-1, 24, 16, 65]              48
            Conv2d-7           [-1, 24, 16, 65]             576
       BatchNorm2d-8           [-1, 24, 16, 65]              48
              ReLU-9           [-1, 24, 16, 65]               0
           Conv2d-10          [-1, 24, 32, 129]             576
      BatchNorm2d-11          [-1, 24, 32, 129]              48
             ReLU-12          [-1, 24, 32, 129]               0
           Conv2d-13           [-1, 24, 16, 65]             216
      BatchNorm2d-14           [-1, 24,

## RGB Dataset
Resnet18 expects RGB input images, so our dataset need to be converted from a one to a three-channel. We'll do dat by copying three times the same image in the three channels.

In [121]:
# Define the custom class for accessing our dataset
class MyDatasetRGB(Dataset):
    def __init__(self, file_list, labels, transform=None, verbose = False):
        self.file_list = file_list
        self.labels=labels
        self.transform = transform
        self.verbose = verbose

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        # returns a training sample and its label
        file_path = self.file_list[idx]
        label = torch.tensor(self.labels[idx])
        stft_vector = torch.tensor(np.load(file_path)) #load from file
        
        # Normalize your data here
        if self.transform:
            if(self.verbose==True):
                print("TRANSFORM: applying transform to tensor shape:",stft_vector.shape,"content:",stft_vector)
            stft_vector = self.transform(torch.unsqueeze(stft_vector, dim=0)) #unsqueeze needed for the torchvision normalize method
            if(self.verbose==True):
                print("TRANSFORM: after transform shape:",stft_vector.shape,"content:",stft_vector)
            stft_vector = torch.squeeze(stft_vector, dim=0)
            if(self.verbose==True):
                print("TRANSFORM: after squeeze shape:",stft_vector.shape,"content:",stft_vector)
                
        #do ResNet18 normalization:
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

        #copy the channel 3 times (need to unsqueeze to create a new dimension first)
        #print("DATASET*  sample shape is:",stft_vector.shape,"content:",stft_vector)
        stft_vector = stft_vector.unsqueeze(0).repeat(3,1,1)
        stft_vector = stft_vector.to(torch.float32) #float32 needed for ResNet18 model (downcast from float64)
        #print("DATASET* sample shape after repeat is:",stft_vector.shape,"content:",stft_vector)
        #print("stft_vector dtype:",stft_vector.dtype)

        
        return stft_vector, label

In [129]:

transform_ResNet18 = transforms.Compose([
    transforms.Normalize(mean= [loaded_mean,loaded_mean,loaded_mean], std=[loaded_std,loaded_std,loaded_std]) #our values
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #ResNet18 specific values
])

train_dataset = MyDatasetRGB(train_file_paths, Y_train,  transform = transform)
validation_dataset = MyDatasetRGB(validation_file_paths, Y_validation,  transform = transform)
test_dataset = MyDatasetRGB(test_file_paths, Y_test,  transform = transform)

train_data_loader = DataLoader(train_dataset, batch_size = 10, shuffle=True)

'''
for batch_idx, batch  in enumerate(train_data_loader):
    print ("batch index:",batch_idx)
    inputs = batch[0]
    labels = batch[1]
    
    for idx, sample in enumerate(inputs):
        label = labels[idx]
        print("inputs: shape:",inputs.shape)
        print("sample: shape:",sample.shape)
'''

'\nfor batch_idx, batch  in enumerate(train_data_loader):\n    print ("batch index:",batch_idx)\n    inputs = batch[0]\n    labels = batch[1]\n    \n    for idx, sample in enumerate(inputs):\n        label = labels[idx]\n        print("inputs: shape:",inputs.shape)\n        print("sample: shape:",sample.shape)\n'

In [156]:
#train the model

#todo: convert input tensor from float64 to double

#todo normalize again using resnet18 suggested mean and std
#TODO: FIX TEST FUNCTION USING RGB (maybe already works)


train_loss_list, train_acc_list, val_loss_list, val_acc_list =train(model, train_dataset, batch_size=128, num_epochs=10, learning_rate=0.001,verbose=False, RGB=True)

  0%|          | 0/1000 [00:00<?, ?batch/s]

KeyboardInterrupt: 